In [1]:
(do 
    (require `serpent.core)
    (require `serpent.pprint)
    (require `serpent.jupyter)
    (serpent.core/configure)
    (add-dependencies [org.mitre.tdp/ttfs-workflow "3.1.25" :exclusions [[net.sf.jsi/jsi]]])
    (set-cda-config :root "tdp:3.1"))

Could not find artifact commons-codec:commons-codec:pom:1.16-20210806.145637-57 in mitre-snapshots (https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots)
Retrieving commons-codec/commons-codec/1.16-SNAPSHOT/commons-codec-1.16-20210806.145637-57.pom (16k)
    from https://repository.apache.org/snapshots/
Retrieving org/mitre/caasd/caasd-project/0.1.3-SNAPSHOT/maven-metadata.xml (1k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots/
Retrieving org/mitre/caasd/jhlabs/proj4-library/1.0.6.1-SNAPSHOT/maven-metadata.xml (1k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots/


{:configuration
 #object[org.apache.hadoop.conf.Configuration 0x718f3d21 "\"Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml\""],
 :meta-repository
 #object[org.mitre.cda.hbase.HBaseMetaRepository 0x1d487096 "\"HBaseMetaRepository[tdp:3.1-meta]\""],
 :hbase-repository
 #object[org.mitre.cda.core.CachingKeyDereferencer 0x32f5d8f0 "\"org.mitre.cda.core.CachingKeyDereferencer@32f5d8f0\""]}


In [2]:
(add-dependencies [org.mitre.tdp/ttfs-procedure "3.1.25" :exclusions [[net.sf.jsi/jsi]]])
(add-dependencies [org.mitre.tdp/tdp-infrastructure "3.1.25" :exclusions [[net.sf.jsi/jsi]]])

("com.google.guava:guava:23.0"
 "net.java.dev.jets3t:jets3t:0.9.4"
 "org.json:json:20140107"
 "org.apache.commons:commons-math3:3.4.1"
 "org.codehaus.mojo:animal-sniffer-annotations:1.14"
 "com.google.errorprone:error_prone_annotations:2.0.18"
 "org.codehaus.jackson:jackson-xc:1.9.13"
 "org.apache.commons:commons-math:2.2"
 "org.scala-lang:scala-reflect:2.11.8"
 "org.apache.httpcomponents:httpcore:4.4.1"
 "com.google.j2objc:j2objc-annotations:1.1"
 "commons-cli:commons-cli:1.2"
 "com.google.code.findbugs:jsr305:1.3.9"
 "org.objenesis:objenesis:2.1"
 "joda-time:joda-time:2.9.4"
 "com.jamesmurty.utils:java-xmlbuilder:1.1"
 "net.iharder:base64:2.3.8"
 "commons-logging:commons-logging:1.2"
 "commons-codec:commons-codec:1.9"
 "org.scala-lang:scala-library:2.11.8"
 ...)


In [3]:
(import org.mitre.caasd.ttfs.ArincTransition
        org.mitre.caasd.ttfs.ArincFix
        org.mitre.caasd.ttfs.ArincAirway
        org.mitre.caasd.ttfs.ArincAirport)

org.mitre.caasd.ttfs.ArincAirport


In [4]:
;; data is small we can just materialize an entire cycle
(def cyc 20210101)

(def transitions (load-data ArincTransition :dates cyc :only "CIFP"))
(def fixes (load-data ArincFix :dates cyc :only "CIFP"))
(def airways (load-data ArincAirway :dates cyc :only "CIFP"))
(def airports (load-data ArincAirport :dates cyc :only "CIFP"))

(str "Transitions: " (count transitions) " Fixes: " (count fixes) " Airways: " (count airways) " Airports: " (count airports))

"Transitions: 61338 Fixes: 67911 Airways: 1550 Airports: 13765"


### Helper Functions

In [5]:
;; Grab all the transitions
(defn null-or-equals [e v]
    (or (= nil v) (= e nil) (= e v)))

(defn proc-filter [pname & kwargs]
    (let [{:keys [airport source] :or {airport nil source nil} :as opts} kwargs]
        (fn [p] (and 
                    (null-or-equals airport ((comp :name :airport) p)) 
                    (null-or-equals pname (:procedure_name p))
                    (null-or-equals source (:navigation_source p))))))

(defn fix-filter [col]
    (fn [f] 
        (some #(= % ((comp :name :identification) f)) col)))

(defn airway-filter [awname & kwargs]
    (let [{:keys [entry exit area-code source]
           :or {entry nil
                exit nil
                area-code nil
                source nil}
           :as opts}
          kwargs]
        (fn [a] (and
                    (null-or-equals awname ((comp :name :identification) a))
                    (some #(null-or-equals entry ((comp :name :identification :path_terminator) %)) (:points a))
                    (some #(null-or-equals exit ((comp :name :identification :path_terminator) %)) (:points a))
                    (null-or-equals area-code ((comp :customer_area_code :arinc_record_info) a))
                    (null-or-equals source ((comp :source :arinc_record_info) a))))))

(defn peek? [ele f] (f ele) ele)

(defn cda-key->string [k] (.toString (.getPrimaryKey k)))

#'user/cda-key->string


### Airway Mocking

In [6]:
(def airwayy (->> airways (filter #(= "Q6" (:identifier %)))))

#'user/airwayy


In [7]:
(defn fixName [leg] (str ((comp :name :identification :path_terminator) leg)))
(defn fixLatLon [leg] (let [latitude ((comp :latitude :path_terminator) leg)
                            longitude ((comp :longitude :path_terminator) leg)]
                          (str " , " latitude " , " longitude)))
(defn toMockitoAirwayBody [airway]
    (let [body (reduce str (->> (:legs airway)
     (map #(str "Leg " (fixName %) "= TF( \"" (fixName %) "\"" (fixLatLon %) "); \n"))))]
    (str body " - primary_key " (cda-key->string (:primary_key airway)) "\n")))

#'user/toMockitoAirwayBody


In [8]:
(->> airwayy (map toMockitoAirwayBody))

("Leg TKA= TF( "TKA" , 62.29837777777777 , -150.10524999999998); 
Leg JOKAP= TF( "JOKAP" , 63.91291111111111 , -150.97479166666668); 
Leg KUTDE= TF( "KUTDE" , 66.32227499999999 , -152.48358055555553); 
Leg LACIL= TF( "LACIL" , 69.504975 , -155.00948055555557); 
Leg BRW= TF( "BRW" , 71.27342777777778 , -156.78811666666667); 
 - primary_key 4-2YeCFS_9AAAJZ
")


### Transition Mocking

#### Procedures By Airport

In [10]:
(def airport-transitions (->> transitions (filter #(= "KATL" ((comp :name :airport) %)))))
(frequencies (map :procedure_name airport-transitions))

{"HAALO2" 9,
 "R27R" 4,
 "H28-Z" 4,
 "NASSA2" 12,
 "L08R" 4,
 "H26RZ" 4,
 "OZZZI1" 14,
 "KAJIN2" 10,
 "PHIIL2" 10,
 "WIGLE2" 7,
 "L10" 4,
 "I26L" 4,
 "GAIRY2" 10,
 "R09L" 4,
 "JJEDI2" 8,
 "R26RY" 4,
 "ZELAN4" 8,
 "R26L" 4,
 "R09R" 4,
 "L27L" 4,
 "R28-Y" 4,
 "PLMMR2" 10,
 "I26R" 4,
 "JACCC2" 10,
 "SITTH2" 11,
 "ATL9" 16,
 "L09R" 4,
 "SMLTZ2" 9,
 "I08R" 4,
 "SINCA6" 6,
 "LGC5" 4,
 "H10-Z" 4,
 "VARNM2" 10,
 "I09L" 4,
 "I08L" 4,
 "L09L" 4,
 "I10" 4,
 "CUTTN2" 10,
 "POUNC2" 11,
 "R10-Y" 4,
 "R27L" 4,
 "SMKEY2" 9,
 "I09R" 4,
 "GLAVN1" 12,
 "WHINZ4" 4,
 "L28" 4,
 "L26L" 4,
 "BANNG2" 9,
 "I27R" 4,
 "CHPPR1" 12,
 "HOBTT2" 14,
 "I27L" 4,
 "L27R" 4,
 "PENCL2" 11,
 "PADGT2" 10,
 "L26R" 4,
 "GNDLF2" 11,
 "I28" 4,
 "L08L" 4,
 "R08R" 4,
 "RMG6" 3,
 "H08LZ" 4,
 "VRSTY2" 9,
 "ONDRE1" 13,
 "R08LY" 4}


In [11]:
(def proc-trans (filter (proc-filter "PLMMR2" :airport "KATL" :source "CIFP") transitions))
(str "total transitions: " (count proc-trans))

"total transitions: 10"


In [12]:
(clojure.string/join "," (map (comp str get-key :primary_key) proc-trans))

"4mIH87B6x_AAGOb,4mIH87B6x_AAGOc,4mIH87B6x_AAGOd,4mIH87B6x_AAGOe,4mIH87B6x_AAGOf,4mIH87B6x_AAGOg,4mIH87B6x_AAGOh,4mIH87B6x_AAGOi,4mIH87B6x_AAGOj,4mIH87B6x_AAGOk"


In [13]:
(map :primary_key (filter #(= "KATL" ((comp :name :identification) %)) airports))

(Key<4pvX1Gw_G4AAB_M>)


In [45]:
(first (load-data org.mitre.caasd.ttfs.NfdcAirport :dates cyc :columns [:primary_key [:icao_identifier = "KMCO"]]))

Record
{:primary_key Key<4Fk9t23D4DAAARe>,
 :icao_identifier "KMCO"}


#### Mocking That Business

In [15]:
(defn toMockitoTransitionBody [transition]
    (reduce str (->> (:legs transition)
     (map #(str "Leg " (fixName %) "=" (:leg_type %) " ( \"" (fixName %) "\"" (fixLatLon %) "); \n")))))

(defn toMockitoTransitionMethod [transition] 
    (let [tname (:transition_name transition)
          pname (:procedure_name transition)
          aname ((comp :name :airport) transition)]
        (str "private static Transition " (:transition_name transition) "(){\n"
         (toMockitoTransitionBody transition)
         "return transition(\"" tname "\",\"" pname "\",\"" aname "\",,,);\n}")))

(defn mockitizeMeCaptain [transitions]
    (reduce str (->> transitions (map #(str (toMockitoTransitionMethod %) "\n")))))

#'user/mockitizeMeCaptain


In [16]:
(mockitizeMeCaptain proc-trans)

"private static Transition RW28(){
Leg =VI ( "" ,  , ); 
Leg WLSON=CF ( "WLSON" , 33.573975000000004 , -84.58462499999999); 
Leg ZALLE=TF ( "ZALLE" , 33.45709166666667 , -84.58449444444445); 
Leg GGOLF=TF ( "GGOLF" , 33.45582777777778 , -84.33613333333332); 
Leg PLMMR=TF ( "PLMMR" , 33.952822222222224 , -83.72700555555556); 
return transition("RW28","PLMMR2","KATL",,,);
}
private static Transition RW10(){
Leg =VI ( "" ,  , ); 
Leg GRITZ=CF ( "GRITZ" , 33.60214444444445 , -84.2763361111111); 
Leg DDUBB=TF ( "DDUBB" , 33.567025 , -84.08201388888888); 
Leg PLMMR=TF ( "PLMMR" , 33.952822222222224 , -83.72700555555556); 
return transition("RW10","PLMMR2","KATL",,,);
}
private static Transition RW09B(){
Leg =VA ( "" ,  , ); 
Leg LIDAS=DF ( "LIDAS" , 33.63363611111111 , -84.26920277777778); 
Leg ERWIN=TF ( "ERWIN" , 33.634166666666665 , -84.10055555555554); 
Leg PLMMR=TF ( "PLMMR" , 33.952822222222224 , -83.72700555555556); 
return transition("RW09B","PLMMR2","KATL",,,);
}
private static Tran